## Step 1 : Dividing the data file into quarter size

Since the data is very huge (2.35 GB) and takes a lot of time to process, we create a chunk of file which is 1/4th the original file

In [ ]:
import os

def get_chunks(file_size):
    chunk_start = 0
    chunk_size = 0x20000
    while chunk_start + chunk_size < file_size:
        yield(chunk_start, chunk_size)
        chunk_start += chunk_size
    final_chunk_size = file_size - chunk_start
    yield(chunk_start, final_chunk_size)

def read_file_chunked(file_path):
    with open(file_path) as file_:
        file_size = os.path.getsize(file_path)
        print file_size
        print('File size: {}'.format(file_size))
        progress = 0

        for chunk_start, chunk_size in get_chunks(file_size):

            file_chunk = file_.read(chunk_size)
            f.write(file_chunk)
            progress += len(file_chunk)
            if(progress>=file_size/2):
                break
            print('{0} of {1} bytes read'.format(progress, file_size))

f = open('prep.tsv', 'ab+')
read_file_chunked('data.tsv')

## Step 2 : Removing the Extra Columns from the Dataset

Originally, the data has 6 fields :
    <user_id> <timestamp> <artist_id> <artist_name> <song_id> <song_name>
From this, we extract only 3 colums to create our MPS :
    <user_id> <timestamp> <song>

In [ ]:
f = open('prep.tsv')
f1 = open('newprep.tsv','w')
for x in xrange(4781164):
    s = f.readline()
    l = s.split('\t')
    f1.write(l[0][5:]+'\t'+l[1]+'\t'+l[-1])
f.close()
f1.close() 

## Step 3 : Creating MPS from the Dataset

Using the Pandas library, we create MPS from the user,timestamp,song values. Songs played between 800s duration are kept in one sequence. MPS with size less than 10 are discarded from the data (As given in the paper)

In [ ]:
import pandas as pd
data = pd.read_csv('newprep.tsv',delimiter='\t',encoding='utf-8')

In [ ]:
data.sort_values(by=['UserID','TimeStamp'],ascending=[True,True],inplace=True)
MPS = {}
idnum = data.iloc[0]['UserID']
song = data.iloc[0]['Song']
ts = data.iloc[0]['TimeStamp']
l = [song]
for i in xrange(1,len(data)):
    newid = data.iloc[i]['UserID']
    tsnew = data.iloc[i]['TimeStamp']
    newsong = data.iloc[i]['Song']
    if newid == idnum:
        d = pd.Timedelta(data.iloc[i]['TimeStamp']-data.iloc[i-1]['TimeStamp']).seconds
        if d < 800:
            l.append(data.iloc[i]['Song'])
        else:
            if idnum in MPS.keys():
                MPS[idnum].append(l)
            else:
                MPS[idnum] = [l]
            l = []
            l.append(data.iloc[i]['Song'])
    else:
        if idnum in MPS.keys():
            MPS[idnum].append(l)
        else:
            MPS[idnum] = [l]
        idnum = newid
        l = []
        l.append(data.iloc[i]['Song'])
if l not in MPS[idnum]:
    if idnum in MPS.keys():
        MPS[idnum].append(l)
    else:
        MPS[idnum] = [l]
for user in MPS:
    for ele in MPS[user]:
        if len(ele) < 10:
            MPS[user].remove(ele)

## Step 4 : Creating a Song2Vec Model using the MPS Data

Now we use the Word2Vec Model implemented in gensim library to create our song2vec model. Here we dont account the songs that have been played less than 10 times (as described in paper) (by setting the 'min_count' variable)

In [ ]:
import gensim.models.word2vec as w2v
import re

In [ ]:
corpus = []
f = open('MPS.txt','r')
for line in f:
    l = line.split("\t")
    l[1] = l[1].replace('nan','')
    l[1] = l[1].replace(', ,',',')
    templist = eval(l[1])
    if len(templist) < 10:
        continue
    corpus.append(templist)
print len(corpus)

135363

In [ ]:
song2vec = w2v.Word2Vec(sg=1,seed=1,size=100,min_count=10,window=5)
song2vec.build_vocab(corpus)
# Preprocess the songs that contain unprintable unicode characters
vocabulary1=list(song2vec.wv.vocab)
print len(vocabulary1)
vocabulary1=list(song2vec.wv.vocab)
vocabulary1=[x.encode('UTF8') for x in vocabulary1]
vocabulary=[]
for word in vocabulary1:
    if re.match('[A-Za-z0-9\'\?\&\)\(\+\!\-\*\.\, ]+$',word):
        vocabulary.append(word)

vocabulary.sort()
print vocabulary
print len(vocabulary)

In [ ]:
song2vec.train(corpus,total_examples=len(corpus),epochs=10)

(32145580, 40475800)

In [ ]:
print song2vec.most_similar(u'The Boy Looked At Johnny',topn=10)

[(u'Radio America', 0.8883671164512634), (u'Music When The Lights Go Out (Demo)', 0.8557537198066711), (u'Cola Queen', 0.8539921045303345), (u'Death On The Stairs (New Recording)', 0.8484163284301758), (u'Babyshambles1', 0.8476840257644653), (u'Albion 1', 0.8459166884422302), (u'The Man Who Would Be King', 0.8445132374763489), (u'The Road To Ruin 2', 0.8408887386322021), (u'Skag & Bone Man', 0.8405758142471313), (u'That Bowery Song', 0.8390020132064819)]


In [ ]:
print song2vec.most_similar('Gang Of Gin',topn=10)

[(u'Fuck Forever (Clean)', 0.86831134557724), (u'Killamangiro', 0.830515444278717), (u'Lust Of The Libertines', 0.8216589689254761), (u'What Katy Did Next', 0.8154677152633667), (u'\xc1 Rebours', 0.8092206120491028), (u'Fuck Forever (Original)', 0.8060870170593262), (u'Beg Steal Or Borrow', 0.8037663102149963), (u'Do You Know Me', 0.8007555603981018), (u'8 Dead Boys', 0.7971910834312439), (u'Fixing Up To Go', 0.7954947352409363)]

In [ ]:
# To find top n songs for any song, just put the name in song name
song_name = ""
l = song2vec.most_similar(song_name,topn=10)
print "Top 10 Songs are: "
for ele in l:
    name,sim = ele[0],ele[1]
    print name

## Step 5 : Creating Playcount Matrix using MPS Data

Creating a matrix containing how many times a particular song was played by user

In [ ]:
play_count = np.zeros((249,len(vocabulary)))
f = open('MPS.txt','r')
for line in f:
    l1 = line.split("\t")
    userno = int(l1[0])
    l1[1] = l1[1].replace('nan','')
    l1[1] = l1[1].replace(', ,',',')
    songs = eval(l1[1])
    if len(songs) < 10:
        continue
    print "User Number :  {0}".format(userno)
    songs=[x.encode('UTF8') for x in songs]
    for song in songs:
        if song in vocabulary:
            index = vocabulary.index(song)
            play_count[userno-1][index] += 1

## Step 6 : Creating the Rating Matrix

We use the playcount matrix to create the rating matrix using the method described in [Choi et. al., 2012]

In [ ]:
AP = np.zeros(play_count.shape)
R = np.zeros(play_count.shape)
for u in xrange(len(play_count)):
    for i in xrange(len(play_count[u])):
        temp = np.sum(play_count[u,:])
        temp1 = np.sum(play_count[:,i])
        AP[u,i] = np.log((temp/temp1)+1)
    print "User {0} Done!".format(u)
for u in xrange(len(play_count)):
    for i in xrange(len(play_count[u])):
        R[u,i] = (5*(AP[u,i]/np.max(AP[:,i])))
    print "User {0} Done!".format(u)

## Step 7 : Matrix Factorization on Ratings Matrix

Now we apply standard matrix factorization on our ratings matrix

In [ ]:
def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in xrange(steps):
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])
                    for k in xrange(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = np.dot(P,Q)
        e = 0
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)
                    for k in xrange(K):
                        e = e + (beta/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) )
        if e < 0.001:
            break
    return P, Q.T


In [ ]:
# Setting the parameters
N = len(R)
M = len(R[0])
K = 100   
iterations = 20
P = np.random.rand(N,K)
Q = np.random.rand(M,K)
nP, nQ = matrix_factorization(R, P, Q, K)
RHat = np.dot(nP,nQ.T)

## Step 8 : Applying Biased Matrix Factorization on Ratings Matrix

Now to compare with the previously applied MF, we apply Biased MF on the ratings Matrix

In [ ]:
class MF():

    def __init__(self, R, K, alpha, beta, iterations):
        """
        Perform matrix factorization to predict empty
        entries in a matrix.

        Arguments
        - R (ndarray)   : user-item rating matrix
        - K (int)       : number of latent dimensions
        - alpha (float) : learning rate
        - beta (float)  : regularization parameter
        """

        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    def train(self):
        # Initialize user and item latent feature matrice
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # Initialize the biases
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])

        # Create a list of training samples
        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]

        # Perform stochastic gradient descent for number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            if (i+1) % 10 == 0:
                print("Iteration: %d ; error = %.4f" % (i+1, mse))

        return training_process

    def mse(self):
        """
        A function to compute the total mean square error
        """
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    def sgd(self):
        """
        Perform stochastic graident descent
        """
        for i, j, r in self.samples:
            # Computer prediction and error
            prediction = self.get_rating(i, j)
            e = (r - prediction)

            # Update biases
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

            # Update user and item latent feature matrices
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    def get_rating(self, i, j):
        """
        Get the predicted rating of user i and item j
        """
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    def full_matrix(self):
        """
        Computer the full matrix using the resultant biases, P and Q
        """
        return self.b + self.b_u[:,np.newaxis] + self.b_i[np.newaxis:,] + self.P.dot(self.Q.T)

In [ ]:
mf = MF(R, K=2, alpha=0.1, beta=0.01, iterations=20)
training_process = mf.train()
RHatBiased = mf.full_matrix()

## Step 9 : Applying the given method in paper to the rating matrix

Using the given loss function in the paper and using the song-song similarities obtained using song2vec to regularize the matrix factorization :

In [ ]:
num_users, num_items = R.shape
b_u_final = np.zeros(num_users)
b_i_final = np.zeros(num_items)
b_final = np.mean(R[np.where(R != 0)])
P_ = []
Q_ = []

In [ ]:
def mse():
    """
    A function to compute the total mean square error
    """
    xs, ys =  R.nonzero()
     b_final +  b_u_final[:,np.newaxis] +  b_i_final[np.newaxis:,] +  P_.dot(Q_.T)
    error = 0
    for x, y in zip(xs, ys):
        error += pow( R[x, y] - predicted[x, y], 2)
    return np.sqrt(error)

In [ ]:
def train():
    global samples, b_u_final, b_i_final, b_final, P_, Q_
    num_users, num_items = R.shape
    lam = 0.1
    P = np.random.normal(scale=1./ K, size=( num_users,  K))
    Q = np.random.normal(scale=1./ K, size=( num_items,  K))
    b_u = np.zeros(num_users)
    b_i = np.zeros(num_items)
    b = np.mean(R[np.where(R != 0)])
    samples = [
        (i, j,  R[i, j])
        for i in range(num_users)
        for j in range(num_items)
        if  R[i, j] > 0]
    training_process = []
    for i in range(iterations):
        np.random.shuffle( samples)
        for l, i, r in samples:
            prediction =  b +  b_u[l] +  b_i[i] +  P[l, :].dot( Q[i, :].T)
            e = (r - prediction)
            # Update biases
            b_u[i] +=  alpha * (e -  beta *  b_u[i])
            b_i[j] +=  alpha * (e -  beta *  b_i[j])
            # Update P Matrix
            P[i, :] +=  alpha * (e *  Q[j, :] -  beta *  P[i,:])
            # Updation of Q based on k nearest neighbours.
            Qi = Q[i,:]
            val = e
            song_name = vocabulary[i]
            top5 = song2vec.most_similar(song_name,topn=5)
            t1 = top5[0][1]
            Qj = Q[vocabulary.index(top[0][0]),:]
            t2 = np.dot(Qi,Qj)
            t3 = (t1-t2)*Qj + lam * Qj
            sub = t3
            for j in xrange(1,5):
                t1 = top[j][1]
                Qj = Q[vocabulary.index(top[j][0]),:]
                t2 = np.dot(Qi,Qj)
                t3 = (t1-t2)*Qj + lam * Qj
                sub += t3
            delQi = val - sub
            Qi = Qi - alpha * (delQi)
            Q[i,:] = Qi
        mse1 =  mse()
        training_process.append((i, mse1))
        if (i+1) % 10 == 0:
            print("Iteration: %d ; error = %.4f" % (i+1, mse))
    b_final = b
    b_u_final = b_u
    b_i_final = b_i
    Q_ = Q
    P_ = P
    return training_process

In [ ]:
errors = train()
RHatPaper = np.dot(P_,Q_.T)